# **Tarik Data E-Purchasing Parquet**
## @ Kurnia Ramadhan @ 
## @ Biro PBJ Setda Prov. Kalbar @
## @ Tahun 2022 @

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install Library System
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=bb4b631e7a302af352be6d9f7038f4cf7bee5c9b7db8bb78bfdec9be77e940ab
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [3]:
# Koneksi ke Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Import Library Penting
import datetime
import os
import wget
import pandas as pd
x = datetime.datetime.now()
#print(x.year)
#print(x.month)
#print(x.day)
#print(x.hour)
#print(x.minute)

In [5]:
# SETTING DIREKTORI & URL API JSON LKPP #
# Sampel: https://inaproc.lkpp.go.id/isb/api/35a90e23-7000-4914-a53b-70ba872dc142/json/180003027/PaketPenyediaOpt1618/tipe/4:12/parameter/2021:D197 

## Kode RUP dan Kode LPSE ##
### Silahkan disesuaikan dengan kode RUP dan kode LPSE dari UKPBJ anda ###
#kodeRUP = "D197"
#kodeLPSE = "97"

## Setting Direktori tempat menyimpan file di Google Drive ##
### Silahkan disesuaikan dengan settingan Google Drive di PC anda ###
direktori = "/content/drive/MyDrive/Coding/Python/Project/dash-ukpbj/data/"
direktori_epurchasing = "/content/drive/MyDrive/Coding/Python/Project/dash-ukpbj/data/epurchasing/"
#direktoricsv = "/content/drive/MyDrive/Coding/Python/Project/dash-ukpbj/data/csv/"
#direktorixlsx = "/content/drive/MyDrive/Coding/Python/Project/dash-ukpbj/data/excel/"
#direktoriftr = "/content/drive/MyDrive/Coding/Python/Project/dash-ukpbj/data/ftr/"

#direktorigabung = "/content/drive/MyDrive/Coding/Python/Project/dash-ukpbj/data/"

## Data eKatalog
### 1 - Data Katalog - Dibiarkan saja
urlTRXKAT = "https://isb.lkpp.go.id/isb/api/c3dee072-3ef1-4b24-8b1d-7239e1ccd7e9/json/736987130/Ecat-PaketEPurchasing/tipe/4:12/parameter/"
urlTRXKATAYANG = "https://isb.lkpp.go.id/isb/api/4bb5a044-79db-4041-9545-acc042ddc1b2/json/736990830/Ecat-ProdukTayangKatalogLokalSektoral/tipe/12/parameter/"
urlTRXKATKOM = "https://isb.lkpp.go.id/isb/api/cd5fecd9-2bab-4b43-8687-98c5f6604319/json/736987127/Ecat-KomoditasDetail/tipe/4/parameter/"

## Data Toko Daring
urlDARING = "https://inaproc.lkpp.go.id/isb/api/4eb548e7-39cc-4ec8-a4db-8c4d702e0e02/json/736989901/TokoDaringRealisasi/tipe/12:4/parameter/"

In [6]:
# Fungsi untuk download data JSON
def unduhJSONTRXKATALOG(jenisdata,kode,tahun):

  if kode == "PROV":
    kodeRUP = "D197"
    kodeLPSE = "97"
  if kode == "SKW":
    kodeRUP = "D200"
    kodeLPSE = "132"
  if kode == "PTK":
    kodeRUP = "D199"
    kodeLPSE = "62"
  if kode == "SBS":
    kodeRUP = "D208"
    kodeLPSE = "354"
  if kode == "MPW":
    kodeRUP = "D552"
    kodeLPSE = "118"
  if kode == "KKR":
    kodeRUP = "D202"
    kodeLPSE = "188"
  if kode == "BKY":
    kodeRUP = "D206"
    kodeLPSE = "444"
  if kode == "LDK":
    kodeRUP = "D205"
    kodeLPSE = "496"  
  if kode == "SGU":
    kodeRUP = "D204"
    kodeLPSE = "298"
  if kode == "SKD":
    kodeRUP = "D198"
    kodeLPSE = "175"
  if kode == "STG":
    kodeRUP = "D211"
    kodeLPSE = "345"
  if kode == "MLW":
    kodeRUP = "D210"
    kodeLPSE = "540"
  if kode == "KPH":
    kodeRUP = "D209"
    kodeLPSE = "488"
  if kode == "KTP":
    kodeRUP = "D201"
    kodeLPSE = "110"
  if kode == "KKU":
    kodeRUP = "D207"
    kodeLPSE = "393"

  if jenisdata == "trxkatalog":
    # Download Data Transaksi Katalog
    kodeJSON = tahun + ":" + kodeRUP
    # Variabel url Data Transaksi Katalog
    urlfile = urlTRXKAT + kodeJSON

  if jenisdata == "prodkatalog":
    # Download Data Produk Katalog Tayang
    kodeJSON = kodeRUP
    # Variabel url Data Produk Tayang
    urlfile = urlTRXKATAYANG + kodeJSON

  if jenisdata == "daring":
    # Download Data Transaksi Toko Daring
    kodeJSON = kodeRUP + ":" + tahun
    # Variabel url Toko Daring
    urlfile = urlDARING + kodeJSON

  fileJSON = direktori + kode + "/" + jenisdata + tahun + ".json"
  fileCSV = direktori + kode + "/" + jenisdata + tahun + ".csv.gz"
  filePARQUET = direktori + kode + "/" + jenisdata + tahun + ".parquet"

  ## Download dana Konversi Data ##
  if tahun != "2022":
    if (os.path.exists(fileCSV) or os.path.exists(filePARQUET)):
      print("Data CSV dan PARQUET tahun " + tahun + " sudah ada, tidak perlu di download lagi !")
    else:
      # Konversi data JSON to CSV, dan PARQUET
      try:
        dfUnduh = pd.read_json(urlfile)
        print("Baca service data " + kode + " - " + jenisdata + " Tahun " + tahun + " Berhasil.")
      except:
        print("Terjadi masalah saat membaca file JSON ", fileJSON)
      try: 
        dfUnduh.to_csv(fileCSV, compression="gzip")
        print("Konversi service data (CSV) " + kode + " - " + jenisdata + " Tahun " + tahun + " Berhasil.")
      except:
        print("Terjadi masalah saat konversi data ke CSV ", fileCSV)
      try:
        dfUnduh.to_parquet(filePARQUET)
        print("Konversi service data (PARQUET) " + kode + " - " + jenisdata + " Tahun " + tahun + " Berhasil.")        
      except:
        print("Terjadi masalah saat konversi data ke PARQUET ", filePARQUET)   
  else:
    if (os.path.exists(fileCSV) or os.path.exists(filePARQUET)):

      # Hapus dulu file data yang sudah ada
      try:
        os.remove(fileCSV, compression="gzip")
      except:
        print("Terjadi masalah saat hapus file ", fileCSV)      
      try:
        os.remove(filePARQUET)
      except:
        print("Terjadi masalah saat hapus file ", filePARQUET)
 
      # Konversi data JSON to CSV, dan PARQUET
      try:
        dfUnduh = pd.read_json(urlfile)
        print("Baca service data " + kode + " - " + jenisdata + " Tahun " + tahun + " Berhasil.")
      except:
        print("Terjadi masalah saat membaca file JSON ", fileJSON)
      try: 
        dfUnduh.to_csv(fileCSV, compression="gzip")
        print("Konversi service data (CSV) " + kode + " - " + jenisdata + " Tahun " + tahun + " Berhasil.")
      except:
        print("Terjadi masalah saat konversi data ke CSV ", fileCSV)
      try:
        dfUnduh.to_parquet(filePARQUET)
        print("Konversi service data (PARQUET) " + kode + " - " + jenisdata + " Tahun " + tahun + " Berhasil.")        
      except:
        print("Terjadi masalah saat konversi data ke PARQUET ", filePARQUET)   

    else:
      # Konversi data JSON to CSV, dan PARQUET
      try:
        dfUnduh = pd.read_json(urlfile)
        print("Baca service data " + kode + " - " + jenisdata + " Tahun " + tahun + " Berhasil.")
      except:
        print("Terjadi masalah saat membaca file JSON ", fileJSON)
      try: 
        dfUnduh.to_csv(fileCSV, compression="gzip")
        print("Konversi service data (CSV) " + kode + " - " + jenisdata + " Tahun " + tahun + " Berhasil.")
      except:
        print("Terjadi masalah saat konversi data ke CSV ", fileCSV)
      try:
        dfUnduh.to_parquet(filePARQUET)
        print("Konversi service data (PARQUET) " + kode + " - " + jenisdata + " Tahun " + tahun + " Berhasil.")        
      except:
        print("Terjadi masalah saat konversi data ke PARQUET ", filePARQUET)   

In [ ]:
pilihs = ["tarikdatakatalog", "gabungdatakatalog", "tarikdatakomoditas", "katalogdetail"]
for pilih in pilihs:

  if pilih == "tarikdatakatalog":

    # Unduh Data E-Purchasing
    kldis = ["PROV", "PTK", "KKR", "MPW", "SKW", "SBS", "BKY", "LDK", "SGU", "SKD", "STG", "MLW", "KPH", "KKU", "KTP"]
    jenisdatas = ["trxkatalog", "prodkatalog", "daring"]
    tahuns = ["2019","2020","2021","2022"]

    #kldi = input("KLDI ? (Format: PROV, PTK, KKR ...")
    #kode = kldi.upper()

    for kode in kldis:
      for jenisdata in jenisdatas:
        for tahun in tahuns:
          unduhJSONTRXKATALOG(jenisdata,kode,tahun)

  elif pilih == "gabungdatakatalog":

    # Gabung Data Transaksi E-Katalog dan Toko Daring
    provs = ["PROV", "PTK", "KKR", "MPW", "SKW", "SBS", "BKY", "LDK", "SGU", "SKD", "STG", "MLW", "KPH", "KKU", "KTP"]

    trxlist = []
    prodlist = []
    tdlist = []

    for prov in provs:
      # Gabung Data Transaksi Katalog
      trxCSV = direktori + prov + "/trxkatalog2022.csv.gz"
      if (os.path.exists(trxCSV)):
        trxkat = pd.read_csv(trxCSV)       
        trxlist.append(trxkat)
      # Gabung Data Produk Katalog
      prodCSV = direktori + prov + "/prodkatalog2022.csv.gz"
      if (os.path.exists(prodCSV)):
        prodkat = pd.read_csv(prodCSV)
        prodlist.append(prodkat)
      # Gabung Data Transaksi Toko Daring
      daringCSV = direktori + prov + "/daring2022.csv.gz"
      if (os.path.exists(daringCSV)):
        trxtd = pd.read_csv(daringCSV)
        tdlist.append(trxtd)
  
    fileKATGABCSV = direktori_epurchasing + "trxkatalog.csv.gz"
    filePRODGABCSV = direktori_epurchasing + "prodkatalog.csv.gz"
    fileDARINGGABCSV = direktori_epurchasing + "daring.csv.gz" 

    fileKATGABPARQUET = direktori_epurchasing + "trxkatalog.parquet"
    filePRODGABPARQUET = direktori_epurchasing + "prodkatalog.parquet"
    fileDARINGGABPARQUET = direktori_epurchasing + "daring.parquet"

    ## Gabung Data Transaksi E-Katalog
    try:
      trxgabung = pd.concat(trxlist)
      print("Penggabungan data E-Katalog Kalbar Sukses")   
    except:
      print("Terjadi masalah saat menggabungkan data E-Katalog Kalbar")
    try:
      trxgabung.to_csv(fileKATGABCSV, compression="gzip")
      print("Konversi Data E-Katalog Kalbar ke format CSV Sukses")
    except:
      print("Terjadi masalah saat konversi Data E-Katalog Kalbar ke format CSV")
    try:
      bacatrxgabung = pd.read_csv(fileKATGABCSV)
      bacatrxgabung.to_parquet(fileKATGABPARQUET)
      print("Konversi Data E-Katalog Kalbar ke format Parquet Sukses")
    except:
      print("Terjadi masalah saat konversi Data E-Katalog Kalbar ke format Parquet")

    ## Gabung Data Produk E-Katalog
    try:
      prodgabung = pd.concat(prodlist)
      print("Penggabungan data Produk E-Katalog Kalbar Sukses")   
    except:
      print("Terjadi masalah saat menggabungkan data Produk E-Katalog Kalbar")
    try:
      prodgabung.to_csv(filePRODGABCSV, compression="gzip")
      print("Konversi Data Produk E-Katalog Kalbar ke format CSV Sukses")
    except:
      print("Terjadi masalah saat konversi Data Produk E-Katalog Kalbar ke format CSV")
    try:
      bacaprodgabung = pd.read_csv(filePRODGABCSV)
      bacaprodgabung.to_parquet(filePRODGABPARQUET)
      print("Konversi Data Produk E-Katalog Kalbar ke format Parquet Sukses")
    except:
      print("Terjadi masalah saat konversi Data Produk E-Katalog Kalbar ke format Parquet")

    ## Gabung Data Transaksi Toko Daring
    try:
      daringgabung = pd.concat(tdlist)
      print("Penggabungan data Transaksi Toko Daring Kalbar Sukses")   
    except:
      print("Terjadi masalah saat menggabungkan data Transaksi Toko Daring Kalbar")
    try:
      daringgabung.to_csv(fileDARINGGABCSV, compression="gzip")
      print("Konversi data Transaksi Toko Daring Kalbar ke format CSV Sukses")
    except:
      print("Terjadi masalah saat konversi data Transaksi Toko Daring Kalbar ke format CSV")      
    try:
      bacadaringgabung = pd.read_csv(fileDARINGGABCSV)
      bacadaringgabung.to_parquet(fileDARINGGABPARQUET)
      print("Konversi data Transaksi Toko Daring Kalbar ke format Parquet Sukses")
    except:
      print("Terjadi masalah saat konversi data Transaksi Toko Daring Kalbar ke format Parquet")

  elif pilih == "tarikdatakomoditas":

    # Tarik Data Komoditas
    trxkatalogdir = direktori_epurchasing + "trxkatalog.csv.gz"
    trxkatalog = pd.read_csv(trxkatalogdir)
    
    fileKOMDETCSV = direktori_epurchasing + "komoditasdetail.csv.gz"
    fileKOMDETPARQUET = direktori_epurchasing + "komoditasdetail.parquet"
    
    kodekomoditas = trxkatalog['kd_komoditas'].unique()
    komoditasgab = []
    for kodekom in kodekomoditas:
      urlkomoditi = urlTRXKATKOM + str(kodekom)
      komoditas = pd.read_json(urlkomoditi)
      komoditasgab.append(komoditas)

    try:
      komoditasdetail = pd.concat(komoditasgab)
      print("Penggabungan data Komoditas Detail Kalbar Sukses")
    except:
      print("Terjadi masalah saat menggabungkan data Komoditas Detail Kalbar")
    try:
      komoditasdetail.to_csv(fileKOMDETCSV, compression="gzip")
      print("Konversi Data Komoditas Detail Kalbar ke format CSV Sukses")
    except:
      print("Terjadi masalah saat konversi Data Komoditas Detail Kalbar ke format CSV")
    try:
      komoditasdetail.to_parquet(fileKOMDETPARQUET)
      print("Konversi Data Komoditas Detail Kalbar ke format Parquet Sukses")
    except:
      print("Terjadi masalah saat konversi Data Komoditas Detail Kalbar ke format Parquet")

  elif pilih == "katalogdetail":

    # Tarik Data Transaksi
    trxkatalogdir = direktori_epurchasing + "trxkatalog.csv.gz"
    trxkatalog = pd.read_csv(trxkatalogdir)

    # Tarik Data Detail Komoditas
    komoditasdetaildir = direktori_epurchasing + "komoditasdetail.csv.gz"
    komoditasdetail = pd.read_csv(komoditasdetaildir)

    fileKATALOGDETAILCSV = direktori_epurchasing + "katalogdetail.csv.gz"
    fileKATALOGDETAILPARQUET = direktori_epurchasing + "katalogdetail.parquet"

    try:
      katalog_detail = trxkatalog.merge(komoditasdetail, how='left', on='kd_komoditas')
      print("Penggabungan data Transaksi dan Detail Komoditas Katalog Sukses")
    except:
      print("Terjadi masalah saat menggabungkan data Katalog Detail Kalbar")
    try:
      katalog_detail.to_csv(fileKATALOGDETAILCSV, compression="gzip")
      print("Konversi Data Katalog Detail Kalbar ke format CSV Sukses")
    except:
      print("Terjadi masalah saat konversi data Katalog Detail CSV Kalbar")
    try:
      katalog_detail.to_parquet(fileKATALOGDETAILPARQUET)
      print("Konversi Data Katalog Detail Kalbar ke format Parquet Sukses")
    except:
      print("Terjadi masalah saat konversi data Katalog Detail Parquet Kalbar")

  else:
    print("Perintah yang anda inginkan tidak tersedia ... !")  